<img src="https://opea.dev/wp-content/uploads/sites/9/2024/04/opea-horizontal-color.svg" alt="OPEA Logo">

# Open Platform for Enterprise AI (OPEA) ChatQnA Blueprint Deployment (DOCKER)

The following notebook will instruct you to deploy a RAG chatbot based on OPEA blueprint ChatQnA blueprints using Docker containers

<div class="warning" style='padding:0.1em; background-color:#FFD700; color:#000000'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>READ BEFORE USING THIS NOTEBOOK</b></p>
<p style='margin-left:1em;'>
Begin at step 2.3 if you already have a pre-provisioned environment. Steps 1 through 2.2 are intended for setting up in a new environment.
</p>
<p style='margin-bottom:1em; margin-right:1em; text-align:right; font-family:Georgia'> <b>-</b> <i></i>
</p></span>
</div>


## Intro : The Open Platform for Enterprise (OPEA) Project 
OPEA uses microservices to create high-quality GenAI applications for enterprises, simplifying the scaling and deployment process for production. These microservices leverage a service composer that assembles them into a megaservice thereby creating real-world enterprise AI applications.

It’s important to familiarize yourself with the MAIN key elements of OPEA:

1. [**GenAIComps**](https://github.com/opea-project/GenAIComps) 
A collection of microservice components that form a service-based toolkit. Each microservice is designed to perform a specific function or task within the GenAI application architecture. By breaking down the system into these smaller, self-contained services, microservices promote modularity, flexibility, and scalability. This modular approach allows developers to independently develop, deploy, and scale individual components of the application, making it easier to maintain and evolve over time. All of the microservices are containerized, allowing cloud native deployment. Here, you will find contributions to multiple partners/communities to further construction.

2. [**GenAIExamples**](https://github.com/opea-project/GenAIExamples)
While *GenAIComps* offers a range of microservices, *GenAIExamples* provides practical, deployable solutions to help users implement these services effectively. This repo provides use-case-based applications that demonstrate how the OPEA architecture can be leveraged to build and deploy real-world GenAI applications. In the repo, developers can find practical resources such as Docker Compose files and Kubernetes Helm charts, which help streamline the deployment and scaling of these applications. These resources allow users to quickly set up and run the examples in local or cloud environments, ensuring a seamless experience.

# 1. Prepare the environment

This notebook uses Docker as an engine, you can deploy this example on any of the available Cloud providers following [THIS](https://opea-project.github.io/latest/getting-started/README.html) guide . 

Once your plaftorm is provisioned, follow the below steps to run the example:.

## 1.1 Install Docker Engine (Linux)

This is the only requirement to have in your environment since OPEA is based on a microservices architecture.

Use the command below to install docker:

In [ ]:
!curl -fsSL https://get.docker.com -o get-docker.sh
!chmod +x get-docker.sh
!sudo sh get-docker.sh --version 26.1
!sudo gpasswd -a $USER docker

## 1.2 Clone GenAIExamples Repo

As mentioned, end-to-end blueprints are provided on OPEA [GenAIExamples repo](https://github.com/opea-project/GenAIExamples), you can see there other examples available like : AgentsQnA, AudioQnA and MultimodalQnA amoung others.

Clone the repo:

In [ ]:
!git clone https://github.com/opea-project/GenAIExamples.git

# 2. Deployment

Let's now explore the OPEA ChatQnA RAG deployment. As mentioned, it's a microservices blueprint designed for scalability, resilience, and flexibility. In this task you will explore each microservice, the purpose of exploring each microservice is to help you understand how each component contributes to the overall application. This learning path will guide you through the system, illustrating the role of each service and how they work together.

Each service can scale individually based on demand, optimizing resources and performance. Additionally, microservices improve fault isolation—if one service fails, it doesn’t disrupt the entire system. This architecture supports efficient maintenance, rapid updates, and adaptability, making it ideal for responding to changing business needs and user demands.

Every OPEA configuration is built on three main parts:

![microservices-arch](./Images/microservices-arch.png)

- **Megaservice** : Microservice "orchestrator". When deploying an end-to-end application with multiple parts involved, there is needed to specify how the flow will be within the microservices. You can learn more from [OPEA documentation](https://github.com/opea-project/GenAIComps?tab=readme-ov-file#megaservice)

- **Gateway** : A gateway is the interface for users to access to the `megaservice` It acts as the entry point for incoming requests, routing them to the appropriate Microservices within the megaservice architecture.

- **Microservice** : Each individual microservice part of the end-to-end application like : **embeddings**, **retrievers**, **LLM** and **vector databases** among others.

You can see below the architecture you will be deploying `ChatQnA`

!["opea-rag"](./Images/opea_rag.png)

## 2.1 Set your enviroment variables 

Each microservice has a set of configurable variables that will specifcy the containter behaviour.For this example, we can specify which Models will be used by the `Embedding`, `LLM` and `reranker`.

## 2.2 Start the containers

OPEA provides containers already created to allow users to build their own blueprint picking each components based on your needs. This is done through a docker compose file, for this example, we will be using the file available on the repo to use `Milvus` as Vector Data base. You can explore some `docker compose` files provisioned for ChatQnA [HERE](https://github.com/opea-project/GenAIExamples/tree/main/ChatQnA/docker_compose), or you can build your own to use the multiple available components on [GenAIComps](https://github.com/opea-project/GenAIComps)

Run `docker compose` to deploy the Blueprint

In [ ]:
%%writefile deploy_chatqna_docker.sh
#!/bin/bash

# Set environment variables
export HUGGINGFACEHUB_API_TOKEN="Your Hugging Face Token"  # Required for Hugging Face authentication
export EMBEDDING_MODEL_ID="BAAI/bge-base-en-v1.5"          # Embedding model ID
export RERANK_MODEL_ID="BAAI/bge-reranker-base"            # Reranker model ID
export LLM_MODEL_ID="Qwen/Qwen3-8B"                        # LLM model ID (e.g., Qwen 8B)
# Uncomment the following line to use a different LLM model
# export LLM_MODEL_ID="meta-llama/Meta-Llama-3-8B-Instruct"
export host_ip=$(hostname -I | awk '{print $1}')            # Get the host IP address

# Set proxy variables if needed (leave empty if not required)
export http_proxy=""
export https_proxy=""
export no_proxy=""

# Source the set_env.sh file if it contains additional environment variables
cd GenAIExamples/ChatQnA/docker_compose/intel/cpu/xeon/
source set_env.sh

# Run Docker Compose with the -E flag to preserve environment variables
# Uncomment the following line to use a different compose file
# sudo -E docker compose -f compose_tgi.yaml up -d
sudo -E docker compose -f compose.yaml up -d

# Print environment variables for debugging
echo "Environment variables set:"
env | grep -E 'HUGGINGFACEHUB_API_TOKEN|MODEL_ID|host_ip|INDEX_NAME|LOGFLAG|proxy'

In [ ]:
# Check if any containers are running
!sudo docker ps

### Note: The cell below deploys all required Docker containers. This process can take 5+ minutes to complete. Please be patient and wait for the cell to finish execution before proceeding.

In [ ]:
%%capture captured_output
!chmod +x deploy_chatqna_docker.sh
!sudo ./deploy_chatqna_docker.sh

## 2.3 Verify Containers

In [ ]:
# You should be able to see 6 containers
!sudo docker ps

This example runs the `LLM` locally, so you might have to wait ~10 min to allow the LLM container to download the model from Hugging Face (this is why you provided the key before).

### Check vLLM Service Connection Status

**IMPORTANT: Re-run this cell repeatedly until you see "completed" in the output. This indicates that the vLLM service is fully initialized and ready for use. Once "completed" appears, the service is ready and you can proceed to the next step**.

**Note: Initial service startup may take 1-2 minutes depending on your environment**


In [ ]:
#Check tgi-service - Wait Until you see tgi-service Conneccted
!sudo docker logs vllm-service 2>&1 | grep complete

# 3. Test your application

In [ ]:
# Run this cell to get your VM's public IP
!curl -s http://ifconfig.me

ChatQnA is an end-to-end blueprint, which means that in addition to the components mentioned before, you will also have an UI to interact with your application.

## 3.1 Use the UI 

## The Role of Nginx in Your Setup

Nginx is functioning as a **reverse proxy** in your architecture. Here's what it's doing:

1. **Front Door**: Nginx listens on port 80 (standard HTTP port) and is the entry point for all web traffic
2. **Traffic Routing**: It receives requests from users and routes them to the appropriate internal services
3. **UI Serving**: It forwards requests to your ChatQnA UI service on port 5173
4. **API Routing**: It routes API calls to your backend services (like the model servers)
5. **Load Balancing**: If you had multiple instances of services, Nginx could distribute traffic between them

## Complete Request Flow

When you access your public IP:

1. Browser makes a request to http://[your-public-ip]/ (implicitly port 80)
2. Request arrives at your VM and is directed to the Nginx container
3. Nginx processes the request based on its configuration
4. For UI requests, Nginx forwards them to the ChatQnA UI container (port 5173)
5. UI makes API calls which Nginx routes to appropriate backend services
6. Response flows back through the same path to your browser

To test on your browser:

1. Copy your Jupyter LAB URL, for example:

`"http://{public_ip}:80`

2. Get your public URL. 

In [ ]:
# Get your public IP address
import subprocess
public_ip = subprocess.check_output("curl -s http://ifconfig.me", shell=True).decode().strip()

# Create the full URL with port 80 (default Nginx port)
url = f"http://{public_ip}:80"

# Display the URL
print(f"WebUI Chat Interface URL: {url}")

4. You should be able to see the public UI
![IMAGE](./Images/chatqna.png)

**NOTE** If you deploy this example in your enviroment, you just need to use your `PUBLIC IP` in your browser

You can now interact with the application, we can ask things like **"What is Nike's revenue in 2023?"**.

Let's now explore each microservice and their behaviour, this will help you understand how they interact and how the RAG example is built

## 3.2 Prompt from the megaservice

Let's first test the `MEGASERVICE` microservice, as mentioned, it's responsible to orchestrate the flow.

## ChatQnA Backend Service Architecture

The ChatQnA backend service functions as the central orchestration layer in the system. Here's its architecture:

### Core Components

1. **API Layer**: Exposes RESTful endpoints like `/v1/chatqna` for client interactions
2. **Session Management**: Maintains conversation history and context between requests
3. **Orchestration Engine**: Coordinates the RAG workflow between services

### Workflow Process

1. When a query arrives, the backend:
   - Processes and sanitizes the input
   - Determines if retrieval is needed
   - Generates embeddings via the TEI service
   - Queries Redis Vector DB for relevant documents
   - Uses reranking to prioritize the most relevant context
   - Constructs a prompt with retrieved context
   - Sends the prompt to TGI for generation
   - Streams the response back to the client

### Integration Points

- **Embedding Service**: For converting queries to vector representations
- **Vector Database**: For semantic document retrieval
- **Language Model**: For generating contextual responses
- **Storage Systems**: For persisting conversations and feedback

The backend service isolates clients from the complexity of the underlying services, presenting a unified interface while handling all the necessary coordination between specialized components.

## Understanding the ChatQnA Architecture and Code

The containers shown in your output represent a sophisticated RAG (Retrieval-Augmented Generation) system with multiple specialized services working together:

- **Nginx** (port 80): Acts as the reverse proxy for all traffic
- **ChatQnA UI** (port 5173): Web interface for user interactions
- **Backend Server** (port 8888): Manages the chat logic and orchestration
- **Text Generation Inference (TGI)** (port 9009): Handles LLM inference
- **Text Embeddings Inference (TEI)** (ports 6006/8808): Creates embeddings and handles reranking
- **Redis Vector DB** (port 6379): Stores vector embeddings for retrieval
- **Retriever and DataPrep services**: Handle document processing and retrieval

This `query_chatqna()` function makes a POST request to the ChatQnA API through Nginx at `/v1/chatqna`. It sends a JSON payload containing the user's query and processes the streamed response. 

In [ ]:
import requests

def query_chatqna(query: str, verbose: bool = True) -> str:
    url = "http://localhost/v1/chatqna"
    headers = {"Content-Type": "application/json"}
    payload = {"messages": query}

    content = ""
    with requests.post(url, json=payload, headers=headers, stream=True) as response:
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode("utf-8")
                if decoded_line == "[DONE]":
                    break
                
                if decoded_line.startswith("data:"):
                    data_part = decoded_line[5:].strip()
                    
                    if data_part.startswith('b"') and data_part.endswith('"'):
                        clean_text = data_part[2:-1]
                    elif data_part.startswith("b'") and data_part.endswith("'"):
                        clean_text = data_part[2:-1]
                    else:
                        clean_text = data_part
                    
                    clean_text = clean_text.replace('\\"', '"').replace("\\'", "'")
                    
                    content += clean_text
                    if verbose:
                        print(clean_text, end="", flush=True)

    return content

You should receive an answer back, verifying that all the services in the RAG flow are working, remember that the microservice is the point of entry and interacts with all the internal services, if you recieve an answer will mean that the blueprint is working.

In [ ]:
# Query ChatQnA without RAG
query = "What was the revenue of Nike in 2023?"
content = query_chatqna(query)


You will see that the model couldn’t answer the question because it lacks the necessary context and operates on ***outdated information***. Without up-to-date or relevant data, it’s unable to provide accurate responses. You'll fix this in few minutes by using RAG to allow the model to pull in current, contextually relevant information, ensuring more precise and relevant answers.

## 3.3 Provide external context before prompting 

Let's help your app to provide context answers!

In this step, you will be feeding the application with context information. From the previous query, the model didn't know **"What is the revenue of Nike in 2023?"**, affortunately, if we have a document containing that information we could feed our application and have it correclty answered. The process then,  is to start from a document (Nike's revenue PDF), and do the preprocessing needed to make it ready to be stored in a database. As shown, this process primarily involves 3 microservices: `data preparation`, `embeddings` and `vector store`. Let's explore each microservice
<p align="center">
  <img src="./Images/pre_flow.png" alt="Alt text" />
</p>

### 3.3.1 Embedding Microservice (Container:chatqna-tei:80)

The document has to be ingested in a way that operations like similarities and storage can be done. Documents aren't stored with words, they are stored as `embeddings`.
An **embedding** is a numerical representation of an object—such as a word, phrase, or document in a continuous vector space. In the context of natural language processing (NLP), embeddings represent words, sentences, or other pieces of text as a set of numbers (or a "vector") that capture their meaning, relationships, and context. By transforming text into this format, embeddings make it easier for machine learning models to understand and work with text data.

For example, the following image shows how word embeddings represent words as points in a vector space based on their relationships. Words with similar meanings, like "king" and "queen" are closer together, and the embedding model captures these connections through vector arithmetic. 

During training, if the model sees "king" often used with "man" and "queen" with "woman," it learns that "king" and "queen" relate similarly to "man" and "woman." So, it positions these words in ways that reflect gender relationships in language.

<p align="center">
  <img src="./Images/king_vs_queen.png" alt="Alt text" />
</p>


Embeddings are a key component for RAG:

•	***Capturing Meaning***: Embeddings represent the semantic relationships between words, allowing RAG models to understand context and nuances in language, enhancing their ability to generate relevant responses.

•	***Dimensionality Reduction***: By converting complex information into fixed-size vectors, embeddings streamline data processing, making RAG systems more efficient and faster.

•	***Improving Model Performance***: Embeddings enable RAG models to generalize better by leveraging semantic similarities, facilitating more accurate information retrieval, and improving the quality of generated content.

OPEA provides multiple options to run your embeddings microservice, as detailed in the [OPEA embedding documentation](https://github.com/opea-project/GenAIComps/tree/main/comps/embeddings): 

In this case, ChatQnA uses [Hugging Face TEI](https://huggingface.co/docs/text-embeddings-inference/en/index) microservice running the embedding model `BAAI/bge-large-en-v1.5` locally. 

Test the microservice by getting the embedding for the phrase "What was Deep Learning?":

This command interfaces with the Text Embedding Interface (TEI) microservice running on port 6006. The service transforms natural language into dense vector representations.

The service converts the query "What is Deep Learning?" into a high-dimensional numerical vector that captures semantic meaning.

In [ ]:
!curl http://localhost:6006/embed -X POST -d \
    '{"inputs":"What is Deep Learning?"}' -H \
    'Content-Type: application/json'

### 3.3.2. Store the public Nike report for 2023 in the Vector Database (chatqna-data-prep:6007)

As mentioned, this example uses Redis as the vector database. You can find all of the supported alternatives in the [OPEA vector store repository](https://github.com/opea-project/GenAIComps/tree/main/comps)

A Vector Database (VDB) is a specialized database designed to store and manage high-dimensional vectors—numeric representations of data points like words, sentences, or images. In AI and machine learning, these vectors are typically embeddings, which capture the meaning and relationships of data in a format that algorithms can process efficiently, as we have shown before.

The Microservice involved on storing the emebddings to the vdb is the `Dataprep` Microservice. It is responsible for preparing data in a digestible format for the application, converting it to embeddings (using the embedding microservice) and loading it to the database. This service preprocesses/transforms the data, making sure it is clean, organized, and suitable for further processing.  

Specifically, this microservice receives data (such as documents), processes it by breaking it into chunks, sends it to the embedding microservice, and stores these vectors in the vector database. The microservice's functionality may depend on the specific vector database being used, as each database has its own requirements for data formatting

To test it and help the model answer the initial question **What was Nike revenue in 2023?**, you will need to upload a context file (revenue report) to be processed. 

1. Execute the following command to download a sample [Nike revenue report](https://github.com/opea-project/GenAIComps/tree/v1.1/comps/retrievers/redis/data) to the nginx pod (if you are no longer logged in to the NGinx pod, be sure to use the above command to log in again):

In [ ]:
# Ingest your data for your RAG. In this case NIke's revenue document

!wget https://raw.githubusercontent.com/opea-project/GenAIComps/v1.1/comps/retrievers/redis/data/nke-10k-2023.pdf

2. Run the ingestion function on the `dataprep` microservice.
This command uploads the document to the DataPrep ingestion service running on port 6007. The service processes the PDF by extracting text, chunking content, generating vector embeddings, and storing these in the vector database—creating searchable knowledge that the ChatQnA system can retrieve during conversations.

In [ ]:
!curl -X POST "http://localhost:6007/v1/dataprep/ingest" \
     -H "Content-Type: multipart/form-data" \
     -F "files=@./nke-10k-2023.pdf"

In [ ]:
!curl -X POST "http://localhost:6007/v1/dataprep/ingest" \
    -H "Content-Type: multipart/form-data" \
    -F "files=@./nke-10k-2023.pdf"

3. Run the get function to retrieve all documents ingested on the vdb

In [ ]:
# You can check if the file was successfully updated
!curl -X POST "http://localhost:6007/v1/dataprep/get" \
     -H "Content-Type: application/json"

## 3.4 Test if your RAG application provide answer based on context (Nike revenue in 2023)

1. Run your query (Directly to the LLM)
This command sends a direct query to the LLM service running on port 9009, bypassing the RAG pipeline. It submits a completion request to the asking a question about uploaded document, with geenrates the response.

In [ ]:
!curl http://localhost:9009/v1/chat/completions \
  -X POST \
  -d '{"model": "'"$LLM_MODEL_ID"'", "messages": [{"role": "user", "content": "What was the revenue of Nike in 2023?"}], "max_tokens":100}' \
  -H 'Content-Type: application/json' | jq -r '.choices[0].message.content'

You can verify that the model doesn't answered correctly, but WHY? 

The answer is simple, the LLM isn't re trained with the context, the information you added in the previous step (Nike revenue) is on the vector database. This information is given as a context (new prompt) to the LLM when is prompted from the entry point `megaservice` which will start the process of recieving the answer, finding the most similar documents (chunks) on the vector database using the `retriever` microservice and finally creating a new prompt that is ingested to the LLM.

2. Query ChatQnA WITH RAG

This code invokes the `query_chatqna()` function to process the question through the complete RAG pipeline. Unlike direct LLM calls, this function handles streaming the response from the ChatQnA service, which retrieves relevant financial data from the vector database, provides it as context to the LLM, and returns a factually-grounded answer about the question asked.

In [ ]:
query = "What was the revenue of Nike in 2023?"
content = query_chatqna(query)

### 3.5.2 Delete documents

In [ ]:
!curl -X POST "http://localhost:6007/v1/dataprep/delete" \
     -d '{"file_path": "all"}' \
     -H "Content-Type: application/json"

### 3.5.3 Delete Containers and Cleanup

In [ ]:
!cd GenAIExamples/ChatQnA/docker_compose/intel/cpu/xeon/ && sudo -E docker compose -f compose.yaml down

In [ ]:
!sudo docker ps

## Summary
The ChatQnA system demonstrates a modern microservices architecture for Retrieval Augmented Generation. By separating text embedding, document ingestion, LLM inference, and orchestration into discrete services, the system achieves modularity and scalability. This architecture allows developers to interact with individual components for specific tasks or leverage the integrated RAG pipeline for knowledge-grounded responses about complex documents like financial reports.

# Be involved with OPEA

OPEA is an open source project that welcomes contributions in many forms. Whether you're interested in fixing bugs, adding new GenAI components, improving documentation, or sharing your unique use cases, there are numerous ways to get involved and make a meaningful impact. 

We’re excited to have you on board and look forward to the contributions you'll bring to the OPEA platform. 

For detailed instructions on how to contribute, please check out our [Contributing Documentation](https://opea-project.github.io/latest/community/CONTRIBUTING.html).

[Follow the project](https://github.com/opea-project) and stay tuned for new events!
